In [1]:
import dmd_with_power_control_single_task as DMD
import imageio
import numpy as np
import time
from glob import glob
import os
import importlib
importlib.reload(DMD)
import napari as nap

In [2]:
XP_FOLDER = r'D:\Data\OR83b_58E02' # 
TARGET_FOLDER = XP_FOLDER + r'\7_1__mVenus_20241125_185125' # 3_1__mVenus_20241124_143220
GENERAL_PARAMETERS = r'C:\\Users\\Light Sheet User\\Desktop\\DMD_parameters.pickled'

mirrors = DMD.dmd(XP_FOLDER, TARGET_FOLDER, GENERAL_PARAMETERS)
# Prints basic information about the DMD.
mirrors.idle()

Loading library: C:/Program Files/ALP-4.3/ALP-4.3 API/x64/alp4395.dll
DMD found, resolution = 1920 x 1200.
DMD patterns folder already exists.


c:\Users\Light Sheet User\Documents\GitHub\DMD\dmd_with_power_control_single_task.py:43: ResourceWarning: unclosed file <_io.BufferedReader name='C:\\\\Users\\\\Light Sheet User\\\\Desktop\\\\DMD_parameters.pickled'>
  self.parameters = pickle.load(open(general_parameters_folder, 'rb'))


SELECT ROIs

In [3]:
mirrors.select_ROIs()

IMPORT the ROIs for USAGE

In [4]:
mirrors.import_N_rois(exp='7_1')

Found 3 ROI(s)


In [ ]:
mirrors.idle()

Learning Protocol - Trace Conditioning - 20 Training Trials, 15s long odor stimulus, 1s ISI Gap, 15s ITI gap

In [5]:
masks_path = r'D:\Data\OR83b_58E02\excitation_patterns\7_1'

# The path to the current acquisition folder is needed in order to save the sequence of projections.
current_acquisition = r'\7_1__DAN_20241125_185643' # D:\Data\
protocol_name = r"/7_1"
images_folder = mirrors.xp_folder + current_acquisition


img1 = imageio.imread(masks_path + '\\mask_1.png') # odour1
img2 = imageio.imread(masks_path + '\\mask_2.png') # odour2
img3 = imageio.imread(masks_path + '\\mask_3.png') # rwdard

start = time.time()

power_odor = 0.1 # changed from 2, 0.1 w/o filter == 2.0 w/ filter
power_rwd = 0.03
steps = []

odor_on_time = 15
rwd_on_time = 10
off_time = 45
train_off_time = 1 # changed from 10
rep_dead_time = 20

steps.append(time.time())
for i in range(2):
    mirrors.project(img1, power_odor); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)
    mirrors.project(img2, power_odor); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)

mirrors.project(img1, power_odor); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)
mirrors.project(img2, power_odor); time.sleep(odor_on_time); mirrors.idle(); time.sleep(60)

steps.append(time.time())

for i in range(19):
    mirrors.project(img1, power_odor); time.sleep(odor_on_time); mirrors.idle(); time.sleep(train_off_time)
    mirrors.project(img3, power_rwd); time.sleep(rwd_on_time); mirrors.idle(); time.sleep(train_off_time)
    mirrors.project(img2, power_odor); time.sleep(odor_on_time); mirrors.idle(); time.sleep(rep_dead_time)

mirrors.project(img1, power_odor); time.sleep(odor_on_time); mirrors.idle(); time.sleep(train_off_time)
mirrors.project(img3, power_rwd); time.sleep(rwd_on_time); mirrors.idle(); time.sleep(train_off_time)
mirrors.project(img2, power_odor); time.sleep(odor_on_time); mirrors.idle(); time.sleep(60)

steps.append(time.time())

for i in range(3):
    mirrors.project(img1, power_odor); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)
    mirrors.project(img2, power_odor); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)

steps.append(time.time())


# SAVE THE PROTOCOL
filenames_camera_0 = r'\SPC00_TM*_ANG000_CM0_CHN00_PH0.stack'
all_volumes = sorted(glob(images_folder + filenames_camera_0))
for num, volume in enumerate(all_volumes):
    current = os.path.getctime(volume)
    if current - start > 0:
        break

timesteps=[]
for t in steps:
    for num, volume in enumerate(all_volumes):
        current = os.path.getctime(volume)
        if current - t > 0:
            timesteps.append(num)
            break


np.savez(images_folder + protocol_name,\
         TM=num, images=["1", "dark", "2", "dark", "1", "dark", "2", "dark", "1", "dark", "2", "dark",
                         "1", "dark", "rwd", "dark", "2", "dark", 
                         "1", "dark", "rwd", "dark", "2", "dark", 
                         "1", "dark", "rwd", "dark", "2", "dark", 
                         "1", "dark", "rwd", "dark", "2", "dark", 
                         "1", "dark", "rwd", "dark", "2", "dark", 
                         "1", "dark", "rwd", "dark", "2", "dark", 
                         "1", "dark", "rwd", "dark", "2", "dark", 
                         "1", "dark", "rwd", "dark", "2", "dark", 
                         "1", "dark", "rwd", "dark", "2", "dark", 
                         "1", "dark", "rwd", "dark", "2", "dark", 
                         "1", "dark", "rwd", "dark", "2", "dark", 
                         "1", "dark", "rwd", "dark", "2", "dark", 
                         "1", "dark", "rwd", "dark", "2", "dark", 
                         "1", "dark", "rwd", "dark", "2", "dark", 
                         "1", "dark", "rwd", "dark", "2", "dark", 
                         "1", "dark", "rwd", "dark", "2", "dark", 
                         "1", "dark", "rwd", "dark", "2", "dark", 
                         "1", "dark", "rwd", "dark", "2", "dark", 
                         "1", "dark", "rwd", "dark", "2", "dark", 
                         "1", "dark", "rwd", "dark", "2", "dark", 
                         "1", "dark", "2", "dark", "1", "dark", "2", "dark", "1", "dark", "2", "dark",
                         ], \
                durations=[odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, 60, 
                           odor_on_time, train_off_time, rwd_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, rwd_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, rwd_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, rwd_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, rwd_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, rwd_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, rwd_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, rwd_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, rwd_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, rwd_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, rwd_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, rwd_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, rwd_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, rwd_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, rwd_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, rwd_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, rwd_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, rwd_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, rwd_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, rwd_on_time, train_off_time, odor_on_time, 60, 
                           odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, off_time, 
                           ]
                ) # 33.6 mins

C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_30296\645133282.py:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img1 = imageio.imread(masks_path + '\\mask_1.png') # odour1
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_30296\645133282.py:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img2 = imageio.imread(masks_path + '\\mask_2.png') # odour2
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_30296\645133282.py:11: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (an

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.030000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.030000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.030000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.030000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.030000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.030000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.030000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.030000 

DMD ON,  input

58E02 Titration - US stimulation

In [ ]:
masks_path = r'D:\Data\58E02_power_titration_george\excitation_patterns\4_1'

# The path to the current acquisition folder is needed in order to save the sequence of projections.
current_acquisition = r'\4_1__DAN_20241123_163626' # D:\Data\g12\
protocol_name = r"/4_1"
images_folder = mirrors.xp_folder + current_acquisition

img1 = imageio.imread(masks_path + '\\mask_1.png') # reward

start = time.time()

powers = [0.1, 0.05, 0.03, 0.01, 0.005] # powers without filter (0.1 w/o filter == 2.0 w/ filter)
steps = []

rwd_on_time = 10
off_time = 45

steps.append(time.time())

for i in range(len(powers)):
    mirrors.project(img1, powers[i]); time.sleep(rwd_on_time); mirrors.idle(); time.sleep(off_time)
    mirrors.project(img1, powers[i]); time.sleep(rwd_on_time); mirrors.idle(); time.sleep(off_time)
    mirrors.project(img1, powers[i]); time.sleep(rwd_on_time); mirrors.idle(); time.sleep(off_time)

steps.append(time.time())

# SAVE THE PROTOCOL
filenames_camera_0 = r'\SPC00_TM*_ANG000_CM0_CHN00_PH0.stack'
all_volumes = sorted(glob(images_folder + filenames_camera_0))
for num, volume in enumerate(all_volumes):
    current = os.path.getctime(volume)
    if current - start > 0:
        break

timesteps=[]
for t in steps:
    for num, volume in enumerate(all_volumes):
        current = os.path.getctime(volume)
        if current - t > 0:
            timesteps.append(num)
            break

np.savez(images_folder + protocol_name,\
         TM=timesteps, images=[
                        "rwd", "dark", "rwd", "dark", "rwd", "dark",
                        "rwd", "dark", "rwd", "dark", "rwd", "dark",
                        "rwd", "dark", "rwd", "dark", "rwd", "dark",
                        "rwd", "dark", "rwd", "dark", "rwd", "dark",
                        "rwd", "dark", "rwd", "dark", "rwd", "dark",
                         ], \
                durations=[
                           rwd_on_time, off_time, rwd_on_time, off_time, rwd_on_time, off_time, 
                           rwd_on_time, off_time, rwd_on_time, off_time, rwd_on_time, off_time, 
                           rwd_on_time, off_time, rwd_on_time, off_time, rwd_on_time, off_time, 
                           rwd_on_time, off_time, rwd_on_time, off_time, rwd_on_time, off_time, 
                           rwd_on_time, off_time, rwd_on_time, off_time, rwd_on_time, off_time, 
                           ]
                ) # 13.75 mins

C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_17840\2912409340.py:8: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img1 = imageio.imread(masks_path + '\\mask_1.png') # reward


DMD ON,  input voltage 0.005000 

DMD ON,  input voltage 0.005000 

DMD ON,  input voltage 0.005000 

DMD ON,  input voltage 0.010000 

DMD ON,  input voltage 0.010000 

DMD ON,  input voltage 0.010000 

DMD ON,  input voltage 0.030000 

DMD ON,  input voltage 0.030000 

DMD ON,  input voltage 0.030000 

DMD ON,  input voltage 0.050000 

DMD ON,  input voltage 0.050000 

DMD ON,  input voltage 0.050000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 



Time Calculation

In [6]:
odor_on_time = 15
rwd_on_time = 10
off_time = 45
train_off_time = 1 # changed from 10
rep_dead_time = 20

durations=[
                           rwd_on_time, off_time, rwd_on_time, off_time, rwd_on_time, off_time, 
                           rwd_on_time, off_time, rwd_on_time, off_time, rwd_on_time, off_time, 
                           rwd_on_time, off_time, rwd_on_time, off_time, rwd_on_time, off_time, 
                           rwd_on_time, off_time, rwd_on_time, off_time, rwd_on_time, off_time, 
                           rwd_on_time, off_time, rwd_on_time, off_time, rwd_on_time, off_time, 
                           ]
t = 0
for i in durations:
    t+= i


print(t/60)

13.75
